In [ ]:
import pickle as pkl
from random import seed

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from conorm import cpm
from matplotlib.patches import Patch
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

seed(1234)
np.random.seed(1234)
torch.random.manual_seed(1234)
from scastral import *

In [2]:
train = pd.read_csv('data/test_set.csv')


KeyboardInterrupt: 